# DSC6135 Summer 2019: HW2

## Import packages of libraries¶


In [1]:
import os
import numpy as np
import warnings

import sklearn.preprocessing
import sklearn.pipeline
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection

from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
%matplotlib inline

## Setup student-defined imports

In [3]:
# None needed

## Load Auto dataset

In [4]:
x_tr_MF = np.loadtxt('data_auto/x_train.csv', delimiter=',', skiprows=1)
x_va_NF = np.loadtxt('data_auto/x_valid.csv', delimiter=',', skiprows=1)
x_te_PF = np.loadtxt('data_auto/x_test.csv', delimiter=',', skiprows=1)

In [5]:
y_tr_M = np.loadtxt('data_auto/y_train.csv', delimiter=',', skiprows=1)
y_va_N = np.loadtxt('data_auto/y_valid.csv', delimiter=',', skiprows=1)
y_te_P = np.loadtxt('data_auto/y_test.csv', delimiter=',', skiprows=1)

## Problem 1: Polynomial Degree Selection on Fixed Validation Set

### **1a**: Plot the number of polynomial features $G$ vs. degree $D$

In [6]:
degree_list = [1, 2, 3, 4, 5, 6, 7, 8]
num_poly_feat_list = []
for degree in degree_list:    
    poly_transformer = sklearn.preprocessing.PolynomialFeatures(
        degree=degree, include_bias=False)
    # TODO make the plot

### 1b: Fit a linear regression model to a polynomial feature transformation of the provided training set of $x$, $y$ values at each of these possible degrees: [1, 2, 3, 4, 5, 6].


In [7]:
degree_list = [1, 2, 3, 4, 5, 6]
err_tr_list = []
err_va_list = []
coef_list = []
for degree in degree_list:
    poly_transformer = sklearn.preprocessing.PolynomialFeatures(
        degree=degree, include_bias=False)
    # TODO fit models and make the plot
    # Be sure to track the coefs/parameters of the learned weights

### **1c:** Plot the Mean Square Error (y-axis) for different polynomial degrees (x-axis) for the train set and validation set. Based on this plot, which single degree value do you recommend? Why?

TODO answer here

### **1d:** Report the numerical values of the 5th percentile and 95th percentile of the coefficients observed in your linear regression model for degrees 3, 4, 5, and 6. What seems to be happening?

In [8]:
# TODO: complete, use np.percentile() function
# to get the percentiles of a numpy array
# (https://docs.scipy.org/doc/numpy/reference/generated/numpy.percentile.html)

The coefficients for high-degree polynomial features are very small (magnitude 10^-8). The values of high-order polynomials are so large that problems occur.

### **1e:** Comment on the training error observed at degree 6. What **should** the training error be at degree 6? What do you think is happening instead?

TODO answer here.

# 2: Linear regression with Rescaled Polynomial Bases

Setup code below to make a pipeline

In [9]:
def make_pipeline(degree=None):
    pipeline = sklearn.pipeline.Pipeline(
        steps=[
         ('rescaler', sklearn.preprocessing.MinMaxScaler()),
         ('poly_transformer', sklearn.preprocessing.PolynomialFeatures(degree=degree, include_bias=False)),
         ('poly_rescaler', sklearn.preprocessing.MinMaxScaler()),
         ('linear_regr', sklearn.linear_model.LinearRegression()),
        ])
    
    # Define utility method that performs all the feature transforms of the pipeline in order.
    def calc_features(x):
        ''' Maps input features to rescaled polynomial features
        '''
        feat = x
        for step_name, step_tfmr in pipeline.steps[:-1]:
            feat = step_tfmr.transform(feat)
        return feat
    pipeline.calc_features = calc_features
    
    # Define utility method that accesses the linear regression weights
    pipeline.get_linear_regr_weights = lambda: pipeline.named_steps['linear_regr'].coef_

    # Return the constructed pipeline
    # We can treat it as if it has a 'regression' API
    # e.g. a fit and a predict method
    return pipeline

In [10]:
degree_list = [1, 2, 3, 4, 5, 6]
err_tr_list = []
err_va_list = []
pipeline_list = []

# TODO fit model at each degree and track error metrics

### 2a: Plot MSE for *rescaled* polynomial features vs. degrees

In [11]:
# TODO make plot here

### **2b:** Using this new analysis, which degree do you recommend? 


ANSWER HERE

# 3: Cross Validation setup

Combine original train/valid splits into one mega dataset

In [12]:
x_trva_LF = np.vstack([x_tr_MF, x_va_NF])
y_trva_L = np.hstack([y_tr_M, y_va_N])

### **Coding Step 1/1:** Complete the starter code function 

In [13]:
def calc_mean_squared_error_across_k_folds(
        model, x_LF, y_L, K=5, random_state=0):
    ''' Calculate mean squared error on K cross-validation folds
    
    Args
    ----
    model: sklearn predictor object has `fit` and `predict` methods
    x_LF : 2D array, size n_examples x n_features
        
    y_L : 1D array, size n_examples
    K : int
    random_state : int or np.random.RandomState instance
    
    Returns
    -------
    mse_scores_K : 1D array, size K
        estimate of MSE on each of the K folds
    '''
    kfold_iterator = sklearn.model_selection.KFold(
        n_splits=K, shuffle=True, random_state=random_state)
    
    # TODO compute mse on each of the K folds

### **3a:** Using your `calc_mean_squared_error_across_k_folds` function with 10 folds, make a line plot of the *average* mean-squared-error at degrees 1, 2, 3, 4, 5, 6. 

In [14]:
# TODO 

### **3b:** Based on this plot, what is your recommended degree? How do your recommendations differ from *Q2*?

**Answer**: TODO

### **3c:** For each $K$, make a scatter plot of the $K$ fold-specific estimates of MSE ($K$ is x-axis, MSE on y-axis). Also draw a line connecting the average MSE across $K$.

In [15]:
degree = 3
K_list = [2, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140]
pipeline = make_pipeline(degree)
err_cv_list = []
avg_score_list = []

# TODO

### **3d:** What happens to the distribution of MSE as you use more and more folds? Is the trend observed in Plot **3c** what we would expect (relevant reading: Chapter 5 at ISL)? 


TODO Answer here.

# 4. Regularized Linear Regression via L2 norm

In [16]:
# regularization hyperparameter: LOSS = MSE + alpha*L2-norm
alpha_grid = np.logspace(-9, 6, 31)

In [17]:
def make_L2regularized_pipeline(degree=2, alpha=1.0):
    ''' This function returns a pipeline object  '''
    pipeline = sklearn.pipeline.Pipeline(
        steps=[
         ('rescaler', sklearn.preprocessing.MinMaxScaler()),
         ('poly_transformer', sklearn.preprocessing.PolynomialFeatures(degree=degree, include_bias=False)),
         ('poly_rescaler', sklearn.preprocessing.MinMaxScaler()),
         ('linear_regr', sklearn.linear_model.Ridge(alpha=alpha)),
        ])
    
    # Define utility method that performs all the feature transforms of the pipeline in order.
    def calc_features(x):
        tmp = x
        for step_name, step_tfmr in pipeline.steps[:-1]:
            tmp = step_tfmr.transform(tmp)
        return tmp
    pipeline.calc_features = calc_features
    
    pipeline.get_linear_regr_weights = lambda: pipeline.named_steps['linear_regr'].coef_
    pipeline.get_linear_regr_bias = lambda: pipeline.named_steps['linear_regr'].intercept_

    return pipeline

In [18]:
# here an auxiliary function to help you in the following questions
def make_pretty_plot_L2_mse_vs_degree(degree=2, alpha_grid=alpha_grid):
    err_tr_list = []
    err_va_list = []
    pipeline_list = []
    for alpha in alpha_grid:
        pipeline = make_L2regularized_pipeline(alpha=alpha, degree=degree)
        pipeline.fit(x_tr_MF, y_tr_M)

        ## TODO: Complete
        ## needs to predict labels and evaluate error in train and test
        # yhat_tr_M = ...
        # yhat_va_N = ...
        #
        # For evaluation, you can use sklearn.metrics.mean_squared_error
        # err_tr = ...
        # err_va = ...
        
        err_tr_list.append(err_tr)
        err_va_list.append(err_va)
        pipeline_list.append(pipeline)
    
    plt.plot(np.log10(alpha_grid), err_tr_list, 'b:', label='train');
    plt.plot(np.log10(alpha_grid), err_va_list, 'rs--', label='valid');
    plt.ylim([0, 70]);
    plt.legend(loc='upper left')
    plt.title('MSE vs. L2 regularization for poly degree = %d' % degree);
    plt.xlabel('Regularization strength scalar alpha');
    plt.ylabel('Mean squared error');

### **4a:** Train `Ridge` regression across grid of possible L2-penalty strengths $\alpha$. Using *degree 2* polynomial features, plot the MSE vs. regularization strength on train and validation.

In [19]:
# TODO plotting code here

### **4b:** Make the same plot at degree 6

In [20]:
# TODO plotting code here

### 4c: How does the recommended $\alpha$ change from deg. 2 to 6?

**Answer**: here